In [1]:
import pandas as pd 
import numpy as np
import datetime as dt

### T0001 - Creating or Altering datatables 

In [2]:
data=pd.read_csv("one_id.csv")
data.columns,data.shape

(Index(['standardized_id', 'dea', 'class_of_trade', 'location_name', 'address',
        'city', 'state', 'zip_code', 'day', 'product_ndc', 'product_name',
        'vaalues_sum', 'row_id', 'batch_id', 'file_id'],
       dtype='object'), (38, 15))

In [3]:
#filtering only latuda drug from the data 
data = data[data['product_name'].str.contains('LATUDA')]
data.columns,data.shape

(Index(['standardized_id', 'dea', 'class_of_trade', 'location_name', 'address',
        'city', 'state', 'zip_code', 'day', 'product_ndc', 'product_name',
        'vaalues_sum', 'row_id', 'batch_id', 'file_id'],
       dtype='object'), (35, 15))

In [4]:
def Standardized_Quantity(df):
    if df.product_name =='LATUDA 20MG 30CT' :
        return(1)
    elif df.product_name =='LATUDA 40MG 30CT' :    
        return(1)
    elif df.product_name =='LATUDA 60MG 30CT' :    
        return(1)
    elif df.product_name =='LATUDA 80MG 30CT' :    
        return(1)
    elif df.product_name =='LATUDA 120MG 30CT' :    
        return(1.5)
    elif df.product_name =='LATUDA 40MG 100CT HUD' :    
        return(1)    
    elif df.product_name =='LATUDA 80MG 100CT HUD' :    
        return(1)
    else:           
        return(0)

In [5]:
data['Standardized_Quantity']=data.apply(Standardized_Quantity,axis=1)
#multiplying quantity with the cost 
data['Drug_Cost_Each_transaction'] = data['Standardized_Quantity']*data['vaalues_sum']

In [6]:
#converting to date format
data['Transaction_Date']=pd.to_datetime(data["day"]).dt.strftime('%Y-%m-%d')

In [7]:
#creating a flag to count number of transactions 
data['Flag']=1

In [8]:
data.shape,data.columns

((35, 19),
 Index(['standardized_id', 'dea', 'class_of_trade', 'location_name', 'address',
        'city', 'state', 'zip_code', 'day', 'product_ndc', 'product_name',
        'vaalues_sum', 'row_id', 'batch_id', 'file_id', 'Standardized_Quantity',
        'Drug_Cost_Each_transaction', 'Transaction_Date', 'Flag'],
       dtype='object'))

In [9]:
#getting min and max transaction date 
min_date=min(data['Transaction_Date'])
max_date=max(data['Transaction_Date'])
print(min_date),print(max_date)

2018-11-07
2020-12-19


(None, None)

In [10]:
#filling dates
idx = pd.date_range(min_date, max_date)
#striping
idx=idx.strftime('%Y-%m-%d')
#for loop to filll the missing dates
standardized_id=[]
Transaction_Date = []
ids = data.standardized_id.unique()
for i in ids:
    for j in idx:
        standardized_id.append(i)
        Transaction_Date.append(j)

In [11]:
#converting to data frame
check=pd.DataFrame({'standardized_id':standardized_id,'Transaction_Date':Transaction_Date})

In [12]:
check.shape

(774, 2)

In [13]:
#Merging with the new table to fill the months 
data1=pd.merge(check,data,on=['standardized_id','Transaction_Date'],how='left')
data1.shape
data1['Year_Month']=pd.to_datetime(data1["Transaction_Date"]).dt.strftime('%Y-%m')
data1=data1.sort_values(by=['standardized_id','Transaction_Date'])
data1.reset_index(inplace=True,drop=True)
data1.shape

(776, 20)

In [176]:
data1=data1.sort_values(by=['standardized_id','Year_Month'])
data1.reset_index(inplace=True,drop=True)

In [177]:
data1.head()

,standardized_id,Transaction_Date,dea,class_of_trade,location_name,address,city,state,zip_code,day,product_ndc,product_name,vaalues_sum,row_id,batch_id,file_id,Standardized_Quantity,Drug_Cost_Each_transaction,Flag,Year_Month
0,100007745,2018-11-07,FJ3131290,Retail Pharmacies,PHILOMATH FAMILY PHARMACY,1640 MAIN ST,PHILOMATH,OR,973709237.0,11/7/2018,6.340203e+10,LATUDA 20MG 30CT,30.0,8.589935e+09,2.020000e+18,33236.0,1.0,30.0,1.0,2018-11
1,100007745,2018-11-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-11
2,100007745,2018-11-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-11
3,100007745,2018-11-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-11
4,100007745,2018-11-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-11


### T0001 - Creating Monthly Lags 

In [178]:
data1.columns

Index(['standardized_id', 'Transaction_Date', 'dea', 'class_of_trade',
       'location_name', 'address', 'city', 'state', 'zip_code', 'day',
       'product_ndc', 'product_name', 'vaalues_sum', 'row_id', 'batch_id',
       'file_id', 'Standardized_Quantity', 'Drug_Cost_Each_transaction',
       'Flag', 'Year_Month'],
      dtype='object')

In [179]:
Monthly_Data=data1.groupby(['standardized_id', 'Year_Month',])['Flag','Drug_Cost_Each_transaction'].sum().reset_index()
Monthly_Data.head()

,standardized_id,Year_Month,Flag,Drug_Cost_Each_transaction
0,100007745,2018-11,2.0,60.0
1,100007745,2018-12,1.0,30.0
2,100007745,2019-01,1.0,90.0
3,100007745,2019-02,2.0,60.0
4,100007745,2019-03,1.0,30.0


In [180]:
Monthly_Data.columns = ['standardized_id', 'Year_Month','NUM_OF_TXNS','STANDARDIZED_QUANTITY']
Monthly_Data.columns

Index(['standardized_id', 'Year_Month', 'NUM_OF_TXNS',
       'STANDARDIZED_QUANTITY'],
      dtype='object')

In [181]:
#creating lags
from tqdm import tqdm
columns = ['NUM_OF_TXNS','STANDARDIZED_QUANTITY']
for i in tqdm(columns):
    feature= i
    #for 6 months 
    Monthly_Data[i+'_1']=Monthly_Data.groupby('standardized_id')[i].shift(1)
    Monthly_Data[i+'_2']=Monthly_Data.groupby('standardized_id')[i+'_1'].shift(1)
    Monthly_Data[i+'_3']=Monthly_Data.groupby('standardized_id')[i+'_2'].shift(1)
    Monthly_Data[i+'_4']=Monthly_Data.groupby('standardized_id')[i+'_3'].shift(1)
    Monthly_Data[i+'_5']=Monthly_Data.groupby('standardized_id')[i+'_4'].shift(1)
    Monthly_Data[i+'_6']=Monthly_Data.groupby('standardized_id')[i+'_5'].shift(1)
    
    #for 12 months 
    Monthly_Data[i+'_7']=Monthly_Data.groupby('standardized_id')[i+'_6'].shift(1)
    Monthly_Data[i+'_8']=Monthly_Data.groupby('standardized_id')[i+'_7'].shift(1)
    Monthly_Data[i+'_9']=Monthly_Data.groupby('standardized_id')[i+'_8'].shift(1)
    Monthly_Data[i+'_10']=Monthly_Data.groupby('standardized_id')[i+'_9'].shift(1)
    Monthly_Data[i+'_11']=Monthly_Data.groupby('standardized_id')[i+'_10'].shift(1)    
    Monthly_Data[i+'_12']=Monthly_Data.groupby('standardized_id')[i+'_11'].shift(1)
    
    #for 24 months 
    Monthly_Data[i+'_13']=Monthly_Data.groupby('standardized_id')[i+'_12'].shift(1)
    Monthly_Data[i+'_14']=Monthly_Data.groupby('standardized_id')[i+'_13'].shift(1)
    Monthly_Data[i+'_15']=Monthly_Data.groupby('standardized_id')[i+'_14'].shift(1)
    Monthly_Data[i+'_16']=Monthly_Data.groupby('standardized_id')[i+'_15'].shift(1)
    Monthly_Data[i+'_17']=Monthly_Data.groupby('standardized_id')[i+'_16'].shift(1)    
    Monthly_Data[i+'_18']=Monthly_Data.groupby('standardized_id')[i+'_17'].shift(1)
    Monthly_Data[i+'_19']=Monthly_Data.groupby('standardized_id')[i+'_18'].shift(1)
    Monthly_Data[i+'_20']=Monthly_Data.groupby('standardized_id')[i+'_19'].shift(1)
    Monthly_Data[i+'_21']=Monthly_Data.groupby('standardized_id')[i+'_20'].shift(1)
    Monthly_Data[i+'_22']=Monthly_Data.groupby('standardized_id')[i+'_21'].shift(1)
    Monthly_Data[i+'_23']=Monthly_Data.groupby('standardized_id')[i+'_22'].shift(1)    
    Monthly_Data[i+'_24']=Monthly_Data.groupby('standardized_id')[i+'_23'].shift(1)    

100%|██████████| 2/2 [00:00<00:00, 24.12it/s]


In [261]:
#Getting Lag fetures for each row
data2=data2.sort_values('Year_Month').groupby('standardized_id').tail(1)

In [262]:
data2.shape

(774, 70)

In [263]:
data2.columns

Index(['standardized_id', 'Transaction_Date', 'dea', 'class_of_trade',
       'location_name', 'address', 'city', 'state', 'zip_code', 'day',
       'product_ndc', 'product_name', 'vaalues_sum', 'row_id', 'batch_id',
       'file_id', 'Standardized_Quantity', 'Drug_Cost_Each_transaction',
       'Flag', 'Year_Month', 'NUM_OF_TXNS', 'STANDARDIZED_QUANTITY',
       'NUM_OF_TXNS_1', 'NUM_OF_TXNS_2', 'NUM_OF_TXNS_3', 'NUM_OF_TXNS_4',
       'NUM_OF_TXNS_5', 'NUM_OF_TXNS_6', 'NUM_OF_TXNS_7', 'NUM_OF_TXNS_8',
       'NUM_OF_TXNS_9', 'NUM_OF_TXNS_10', 'NUM_OF_TXNS_11', 'NUM_OF_TXNS_12',
       'NUM_OF_TXNS_13', 'NUM_OF_TXNS_14', 'NUM_OF_TXNS_15', 'NUM_OF_TXNS_16',
       'NUM_OF_TXNS_17', 'NUM_OF_TXNS_18', 'NUM_OF_TXNS_19', 'NUM_OF_TXNS_20',
       'NUM_OF_TXNS_21', 'NUM_OF_TXNS_22', 'NUM_OF_TXNS_23', 'NUM_OF_TXNS_24',
       'STANDARDIZED_QUANTITY_1', 'STANDARDIZED_QUANTITY_2',
       'STANDARDIZED_QUANTITY_3', 'STANDARDIZED_QUANTITY_4',
       'STANDARDIZED_QUANTITY_5', 'STANDARDIZED_QUANTI

In [264]:
data3=data2[['standardized_id', 'Transaction_Date', 'dea', 'class_of_trade',
       'location_name', 'address', 'city', 'state', 'zip_code', 'day',
       'product_ndc', 'product_name', 'vaalues_sum', 'row_id', 'batch_id',
       'file_id', 'Standardized_Quantity', 'Drug_Cost_Each_transaction',
       'Flag', 'Year_Month', 'NUM_OF_TXNS', 'STANDARDIZED_QUANTITY',
       'NUM_OF_TXNS_1', 'NUM_OF_TXNS_2', 'NUM_OF_TXNS_3', 'NUM_OF_TXNS_4',
       'NUM_OF_TXNS_5', 'NUM_OF_TXNS_6', 'NUM_OF_TXNS_7', 'NUM_OF_TXNS_8',
       'NUM_OF_TXNS_9', 'NUM_OF_TXNS_10', 'NUM_OF_TXNS_11', 'NUM_OF_TXNS_12',
       'NUM_OF_TXNS_13', 'NUM_OF_TXNS_14', 'NUM_OF_TXNS_15', 'NUM_OF_TXNS_16',
       'NUM_OF_TXNS_17', 'NUM_OF_TXNS_18', 'NUM_OF_TXNS_19', 'NUM_OF_TXNS_20',
       'NUM_OF_TXNS_21', 'NUM_OF_TXNS_22', 'NUM_OF_TXNS_23', 'NUM_OF_TXNS_24',
       'STANDARDIZED_QUANTITY_1', 'STANDARDIZED_QUANTITY_2',
       'STANDARDIZED_QUANTITY_3', 'STANDARDIZED_QUANTITY_4',
       'STANDARDIZED_QUANTITY_5', 'STANDARDIZED_QUANTITY_6',
       'STANDARDIZED_QUANTITY_7', 'STANDARDIZED_QUANTITY_8',
       'STANDARDIZED_QUANTITY_9', 'STANDARDIZED_QUANTITY_10',
       'STANDARDIZED_QUANTITY_11', 'STANDARDIZED_QUANTITY_12',
       'STANDARDIZED_QUANTITY_13', 'STANDARDIZED_QUANTITY_14',
       'STANDARDIZED_QUANTITY_15', 'STANDARDIZED_QUANTITY_16',
       'STANDARDIZED_QUANTITY_17', 'STANDARDIZED_QUANTITY_18',
       'STANDARDIZED_QUANTITY_19', 'STANDARDIZED_QUANTITY_20',
       'STANDARDIZED_QUANTITY_21', 'STANDARDIZED_QUANTITY_22',
       'STANDARDIZED_QUANTITY_23', 'STANDARDIZED_QUANTITY_24']]

In [265]:
data3.columns

Index(['standardized_id', 'Transaction_Date', 'dea', 'class_of_trade',
       'location_name', 'address', 'city', 'state', 'zip_code', 'day',
       'product_ndc', 'product_name', 'vaalues_sum', 'row_id', 'batch_id',
       'file_id', 'Standardized_Quantity', 'Drug_Cost_Each_transaction',
       'Flag', 'Year_Month', 'NUM_OF_TXNS', 'STANDARDIZED_QUANTITY',
       'NUM_OF_TXNS_1', 'NUM_OF_TXNS_2', 'NUM_OF_TXNS_3', 'NUM_OF_TXNS_4',
       'NUM_OF_TXNS_5', 'NUM_OF_TXNS_6', 'NUM_OF_TXNS_7', 'NUM_OF_TXNS_8',
       'NUM_OF_TXNS_9', 'NUM_OF_TXNS_10', 'NUM_OF_TXNS_11', 'NUM_OF_TXNS_12',
       'NUM_OF_TXNS_13', 'NUM_OF_TXNS_14', 'NUM_OF_TXNS_15', 'NUM_OF_TXNS_16',
       'NUM_OF_TXNS_17', 'NUM_OF_TXNS_18', 'NUM_OF_TXNS_19', 'NUM_OF_TXNS_20',
       'NUM_OF_TXNS_21', 'NUM_OF_TXNS_22', 'NUM_OF_TXNS_23', 'NUM_OF_TXNS_24',
       'STANDARDIZED_QUANTITY_1', 'STANDARDIZED_QUANTITY_2',
       'STANDARDIZED_QUANTITY_3', 'STANDARDIZED_QUANTITY_4',
       'STANDARDIZED_QUANTITY_5', 'STANDARDIZED_QUANTI

### End of Creating data table (one time thing)

### T0002 - Data operations on the master dataset

In [187]:
data3.head()

,standardized_id,Year_Month,NUM_OF_TXNS,STANDARDIZED_QUANTITY,NUM_OF_TXNS_1,NUM_OF_TXNS_2,NUM_OF_TXNS_3,NUM_OF_TXNS_4,NUM_OF_TXNS_5,NUM_OF_TXNS_6,...,STANDARDIZED_QUANTITY_15,STANDARDIZED_QUANTITY_16,STANDARDIZED_QUANTITY_17,STANDARDIZED_QUANTITY_18,STANDARDIZED_QUANTITY_19,STANDARDIZED_QUANTITY_20,STANDARDIZED_QUANTITY_21,STANDARDIZED_QUANTITY_22,STANDARDIZED_QUANTITY_23,STANDARDIZED_QUANTITY_24
25,100007745,2020-12,1.0,30.0,0.0,2.0,0.0,1.0,1.0,1.0,...,30.0,60.0,90.0,90.0,0.0,90.0,30.0,60.0,90.0,30.0


In [203]:
data_update_check=pd.read_csv("delete.csv")

In [204]:
data_update_check.columns

Index(['standardized_id', 'dea', 'class_of_trade', 'location_name', 'address',
       'city', 'state', 'zip_code', 'day', 'product_ndc', 'product_name',
       'vaalues_sum', 'row_id', 'batch_id', 'file_id'],
      dtype='object')

In [205]:
data_update_check.shape

(1, 15)

In [315]:
Current_months = data3['Year_Month'].unique()
ids = data_update_check['row_id'].unique()
#looks for all rows considering rowid or transaction id 
for i in tqdm(ids):
    #filtering for the drug 
    df=data_update_check[data_update_check['product_name'].str.contains('LATUDA')]
    #getting year month from day 
    df['Year_Month']=pd.to_datetime(df["day"]).dt.strftime('%Y-%m')
    #filtering to check if it is a new or old record
    temp=df[~df['Year_Month'].isin(Current_months)]
    #checking the length of the record
    check=len(temp)
    #checking if the length of the row is greater than 0 
    if(check>0):
        print('There is a record')
        temp1=df['standardized_id'].iloc[0]
        print("STANDARDIZED_ID",temp1)
        
        #Filtering all the rows based on that ID 
        temp2=data_update_check[data_update_check['standardized_id']==temp1]
        print("COLUMNS",temp2.columns)
        
        #getting year month from day 
        temp2['Year_Month']=pd.to_datetime(temp2["day"]).dt.strftime('%Y-%m')
        print("YEAR_MON",temp2.Year_Month)
        
        #filtering to check if it is a new record
        temp3=temp2[~temp2['Year_Month'].isin(Current_months)]
        print('ITS A NEW RECORD')
        
        temp3['Standardized_Quantity1']=temp3.apply(Standardized_Quantity,axis=1)
        print("STANDARDIZED_QUANTITY",temp3.Standardized_Quantity1)
        
        #multiplying quantity with the cost 
        temp3['Drug_Cost_Each_transaction'] = temp3['Standardized_Quantity1']*temp3['vaalues_sum']
        print("Drug_Cost_Each_transaction",temp3.Drug_Cost_Each_transaction)
        
        #converting to date format
        temp3['Transaction_Date']=pd.to_datetime(temp3["day"]).dt.strftime('%Y-%m-%d')
        print("Transaction_Date",temp3.Transaction_Date)
        #creating a flag to count number of transactions 
        temp3['Flag']=1
        print("Flag",temp3.Flag)
        
        
        temp4=temp3.groupby(['standardized_id', 'Year_Month',])['Flag','Drug_Cost_Each_transaction'].sum().reset_index()
        temp4.columns = ['standardized_id', 'Year_Month','NUM_OF_TXNS','STANDARDIZED_QUANTITY']
        print(temp4.columns)
        
        #Getting Lag fetures for each row
        temp5=pd.merge(temp3,temp4,on=['standardized_id','Year_Month'],how='left')
        print(temp5.columns)
        
        temp6=data3[data3['standardized_id']==temp1]
        temp6=temp6.sort_values('Year_Month').groupby('standardized_id').tail(1)
        print(temp6.shape)
        
        temp5['NUM_OF_TXNS1']=temp6['NUM_OF_TXNS'].iloc[0]
        temp5['NUM_OF_TXNS2']=temp6['NUM_OF_TXNS_1'].iloc[0]
        temp5['NUM_OF_TXNS3']=temp6['NUM_OF_TXNS_2'].iloc[0]
        temp5['NUM_OF_TXNS4']=temp6['NUM_OF_TXNS_3'].iloc[0]
        temp5['NUM_OF_TXNS5']=temp6['NUM_OF_TXNS_4'].iloc[0]
        temp5['NUM_OF_TXNS6']=temp6['NUM_OF_TXNS_5'].iloc[0]
        temp5['NUM_OF_TXNS7']=temp6['NUM_OF_TXNS_6'].iloc[0]
        
        temp5['NUM_OF_TXNS8']=temp6['NUM_OF_TXNS_7'].iloc[0]
        temp5['NUM_OF_TXNS9']=temp6['NUM_OF_TXNS_8'].iloc[0]
        temp5['NUM_OF_TXNS10']=temp6['NUM_OF_TXNS_9'].iloc[0]
        temp5['NUM_OF_TXNS11']=temp6['NUM_OF_TXNS_10'].iloc[0]
        temp5['NUM_OF_TXNS12']=temp6['NUM_OF_TXNS_11'].iloc[0]
        temp5['NUM_OF_TXNS13']=temp6['NUM_OF_TXNS_12'].iloc[0]
        temp5['NUM_OF_TXNS14']=temp6['NUM_OF_TXNS_13'].iloc[0]
        
        temp5['NUM_OF_TXNS15']=temp6['NUM_OF_TXNS_14'].iloc[0]
        temp5['NUM_OF_TXNS16']=temp6['NUM_OF_TXNS_15'].iloc[0]
        temp5['NUM_OF_TXNS17']=temp6['NUM_OF_TXNS_16'].iloc[0]
        temp5['NUM_OF_TXNS18']=temp6['NUM_OF_TXNS_17'].iloc[0]
        temp5['NUM_OF_TXNS19']=temp6['NUM_OF_TXNS_18'].iloc[0]
        temp5['NUM_OF_TXNS20']=temp6['NUM_OF_TXNS_19'].iloc[0]
        temp5['NUM_OF_TXNS21']=temp6['NUM_OF_TXNS_20'].iloc[0]
        
        temp5['NUM_OF_TXNS22']=temp6['NUM_OF_TXNS_21'].iloc[0]
        temp5['NUM_OF_TXNS23']=temp6['NUM_OF_TXNS_22'].iloc[0]
        temp5['NUM_OF_TXNS24']=temp6['NUM_OF_TXNS_23'].iloc[0]
        
    else:
        print('There is no record')

  0%|          | 0/1 [00:00<?, ?it/s]

There is a record
STANDARDIZED_ID 100007745
COLUMNS Index(['standardized_id', 'dea', 'class_of_trade', 'location_name', 'address',
       'city', 'state', 'zip_code', 'day', 'product_ndc', 'product_name',
       'vaalues_sum', 'row_id', 'batch_id', 'file_id'],
      dtype='object')
YEAR_MON 0    2021-11
Name: Year_Month, dtype: object
ITS A NEW RECORD
STANDARDIZED_QUANTITY 0    1
Name: Standardized_Quantity1, dtype: int64
Drug_Cost_Each_transaction 0    30
Name: Drug_Cost_Each_transaction, dtype: int64
Transaction_Date 0    2021-11-07
Name: Transaction_Date, dtype: object
Flag 0    1
Name: Flag, dtype: int64
Index(['standardized_id', 'Year_Month', 'NUM_OF_TXNS',
       'STANDARDIZED_QUANTITY'],
      dtype='object')
Index(['standardized_id', 'dea', 'class_of_trade', 'location_name', 'address',
       'city', 'state', 'zip_code', 'day', 'product_ndc', 'product_name',
       'vaalues_sum', 'row_id', 'batch_id', 'file_id', 'Year_Month',
       'Standardized_Quantity1', 'Drug_Cost_Each_tra

100%|██████████| 1/1 [00:00<00:00,  8.36it/s]


In [323]:
temp5['Transaction_Date']

0    2021-11-07
Name: Transaction_Date, dtype: object

In [284]:
temp6

,standardized_id,Transaction_Date,dea,class_of_trade,location_name,address,city,state,zip_code,day,...,STANDARDIZED_QUANTITY_15,STANDARDIZED_QUANTITY_16,STANDARDIZED_QUANTITY_17,STANDARDIZED_QUANTITY_18,STANDARDIZED_QUANTITY_19,STANDARDIZED_QUANTITY_20,STANDARDIZED_QUANTITY_21,STANDARDIZED_QUANTITY_22,STANDARDIZED_QUANTITY_23,STANDARDIZED_QUANTITY_24
773,100007745,2020-12-19,FJ3131290,Retail Pharmacies,PHILOMATH FAMILY PHARMACY,1640 MAIN ST,PHILOMATH,OR,973709237.0,12/19/2020,...,30.0,60.0,90.0,90.0,0.0,90.0,30.0,60.0,90.0,30.0


In [318]:
temp5.to_csv("file.csv",index=False)

In [357]:
Current_months = data3['Year_Month'].unique()
ids = data_update_check['row_id'].unique()
#looks for all rows considering rowid or transaction id 
for i in tqdm(ids):
    #filtering for the drug 
    df=data_update_check[data_update_check['product_name'].str.contains('LATUDA')]
    #getting year month from day 
    df['Year_Month']=pd.to_datetime(df["day"]).dt.strftime('%Y-%m')
    #filtering to check if it is a new or old record
    temp=df[~df['Year_Month'].isin(Current_months)]
    #checking the length of the record
    check=len(temp)
    #checking if the length of the row is greater than 0 
    if(check>0):
        print('There is a record')
        temp1=df['standardized_id'].iloc[0]
        print("STANDARDIZED_ID",temp1)
        
        #Filtering all the rows based on that ID 
        temp2=data_update_check[data_update_check['standardized_id']==temp1]
        print("COLUMNS",temp2.columns)
        
        #getting year month from day 
        temp2['Year_Month']=pd.to_datetime(temp2["day"]).dt.strftime('%Y-%m')
        print("YEAR_MON",temp2.Year_Month)
        
        #filtering to check if it is a new record
        temp3=temp2[~temp2['Year_Month'].isin(Current_months)]
        print('ITS A NEW RECORD')
        
        temp3['Standardized_Quantity1']=temp3.apply(Standardized_Quantity,axis=1)
        print("STANDARDIZED_QUANTITY",temp3.Standardized_Quantity1)
        
        #multiplying quantity with the cost 
        temp3['Drug_Cost_Each_transaction'] = temp3['Standardized_Quantity1']*temp3['vaalues_sum']
        print("Drug_Cost_Each_transaction",temp3.Drug_Cost_Each_transaction)
        
        #converting to date format
        temp3['Transaction_Date']=pd.to_datetime(temp3["day"]).dt.strftime('%Y-%m-%d')
        print("Transaction_Date",temp3.Transaction_Date)
        #creating a flag to count number of transactions 
        temp3['Flag']=1
        print("Flag",temp3.Flag)
        
        
        temp4=temp3.groupby(['standardized_id', 'Year_Month',])['Flag','Drug_Cost_Each_transaction'].sum().reset_index()
        temp4.columns = ['standardized_id', 'Year_Month','NUM_OF_TXNS','STANDARDIZED_QUANTITY']
        print(temp4.columns)
        
        #Getting Lag fetures for each row
        temp5=pd.merge(temp3,temp4,on=['standardized_id','Year_Month'],how='left')
        print(temp5.columns)
        
        
        temp6=data3[data3['standardized_id']==temp1]
        temp6=temp6.sort_values('Year_Month').groupby('standardized_id').tail(1)
        print(temp6.shape)
        
        temp5['NUM_OF_TXNS1']=temp6['NUM_OF_TXNS'].iloc[0]
        temp5['NUM_OF_TXNS2']=temp6['NUM_OF_TXNS_1'].iloc[0]
        temp5['NUM_OF_TXNS3']=temp6['NUM_OF_TXNS_2'].iloc[0]
        temp5['NUM_OF_TXNS4']=temp6['NUM_OF_TXNS_3'].iloc[0]
        temp5['NUM_OF_TXNS5']=temp6['NUM_OF_TXNS_4'].iloc[0]
        temp5['NUM_OF_TXNS6']=temp6['NUM_OF_TXNS_5'].iloc[0]
        temp5['NUM_OF_TXNS7']=temp6['NUM_OF_TXNS_6'].iloc[0]
        
        temp5['NUM_OF_TXNS8']=temp6['NUM_OF_TXNS_7'].iloc[0]
        temp5['NUM_OF_TXNS9']=temp6['NUM_OF_TXNS_8'].iloc[0]
        temp5['NUM_OF_TXNS10']=temp6['NUM_OF_TXNS_9'].iloc[0]
        temp5['NUM_OF_TXNS11']=temp6['NUM_OF_TXNS_10'].iloc[0]
        temp5['NUM_OF_TXNS12']=temp6['NUM_OF_TXNS_11'].iloc[0]
        temp5['NUM_OF_TXNS13']=temp6['NUM_OF_TXNS_12'].iloc[0]
        temp5['NUM_OF_TXNS14']=temp6['NUM_OF_TXNS_13'].iloc[0]
        
        temp5['NUM_OF_TXNS15']=temp6['NUM_OF_TXNS_14'].iloc[0]
        temp5['NUM_OF_TXNS16']=temp6['NUM_OF_TXNS_15'].iloc[0]
        temp5['NUM_OF_TXNS17']=temp6['NUM_OF_TXNS_16'].iloc[0]
        temp5['NUM_OF_TXNS18']=temp6['NUM_OF_TXNS_17'].iloc[0]
        temp5['NUM_OF_TXNS19']=temp6['NUM_OF_TXNS_18'].iloc[0]
        temp5['NUM_OF_TXNS20']=temp6['NUM_OF_TXNS_19'].iloc[0]
        temp5['NUM_OF_TXNS21']=temp6['NUM_OF_TXNS_20'].iloc[0]
        
        temp5['NUM_OF_TXNS22']=temp6['NUM_OF_TXNS_21'].iloc[0]
        temp5['NUM_OF_TXNS23']=temp6['NUM_OF_TXNS_22'].iloc[0]
        temp5['NUM_OF_TXNS24']=temp6['NUM_OF_TXNS_23'].iloc[0]
        
    else:
        print('There is no record')

  0%|          | 0/1 [00:00<?, ?it/s]

There is a record
STANDARDIZED_ID 100007745
COLUMNS Index(['standardized_id', 'dea', 'class_of_trade', 'location_name', 'address',
       'city', 'state', 'zip_code', 'day', 'product_ndc', 'product_name',
       'vaalues_sum', 'row_id', 'batch_id', 'file_id'],
      dtype='object')
YEAR_MON 0    2021-11
Name: Year_Month, dtype: object
ITS A NEW RECORD
STANDARDIZED_QUANTITY 0    1
Name: Standardized_Quantity1, dtype: int64
Drug_Cost_Each_transaction 0    30
Name: Drug_Cost_Each_transaction, dtype: int64
Transaction_Date 0    2021-11-07
Name: Transaction_Date, dtype: object
Flag 0    1
Name: Flag, dtype: int64
Index(['standardized_id', 'Year_Month', 'NUM_OF_TXNS',
       'STANDARDIZED_QUANTITY'],
      dtype='object')
Index(['standardized_id', 'dea', 'class_of_trade', 'location_name', 'address',
       'city', 'state', 'zip_code', 'day', 'product_ndc', 'product_name',
       'vaalues_sum', 'row_id', 'batch_id', 'file_id', 'Year_Month',
       'Standardized_Quantity1', 'Drug_Cost_Each_tra

100%|██████████| 1/1 [00:00<00:00,  7.83it/s]


In [321]:
temp6.Transaction_Date-temp5.Transaction_Date

0      NaN
773    NaN
Name: Transaction_Date, dtype: object

In [348]:
((temp6.Transaction_Date-temp5.Transaction_Date)/np.timedelta64(1, 'M'))

0     NaN
773   NaN
Name: Transaction_Date, dtype: float64

In [346]:
num_months = (temp5.Transaction_Date - temp6.Transaction_Date) * 12 + (temp5.Transaction_Date - temp6.Transaction_Date)

In [347]:
num_months

0     NaT
773   NaT
Name: Transaction_Date, dtype: timedelta64[ns]

In [330]:
temp5.Transaction_Date

0    2021-11-07
Name: Transaction_Date, dtype: object

In [331]:
temp6.Transaction_Date

773    2020-12-19
Name: Transaction_Date, dtype: object

In [340]:
temp5['Transaction_Date'] - temp6['Transaction_Date']

0      NaN
773    NaN
Name: Transaction_Date, dtype: object

In [341]:
pd.to_datetime(temp5["Transaction_Date"],format='%Y-%m-%d')

0   2021-11-07
Name: Transaction_Date, dtype: datetime64[ns]

In [359]:
temp6['Transaction_Date']

773    2020-12-19
Name: Transaction_Date, dtype: object

In [360]:
temp5['Transaction_Date']

0    2021-11-07
Name: Transaction_Date, dtype: object

In [349]:
pd.to_datetime(temp6["Transaction_Date"],format='%Y-%m-%d') - pd.to_datetime(temp5["Transaction_Date"],format='%Y-%m-%d')

0     NaT
773   NaT
Name: Transaction_Date, dtype: timedelta64[ns]

In [350]:
dict(temp6["Transaction_Date"],temp5["Transaction_Date"])

TypeError: dict expected at most 1 arguments, got 2

In [358]:
(temp6["Transaction_Date"] - temp5["Transaction_Date"]).dt.days    

AttributeError: Can only use .dt accessor with datetimelike values

In [352]:
pd.DataFrame(pd.date_range('2014-11-19', '2014-11-21', freq='D'))

,0
0,2014-11-19
1,2014-11-20
2,2014-11-21


In [355]:
temp5["Transaction_Date"] -  pd.to_timedelta(temp6["Transaction_Date"], unit='d')

C:\ProgramData\Anaconda3\lib\site-packages\pandas\util\_decorators.py:208: FutureWarning: Passing datetime64-dtype data to TimedeltaIndex is deprecated, will raise a TypeError in a future version
  return func(*args, **kwargs)


0     NaT
773   NaT
Name: Transaction_Date, dtype: datetime64[ns]

In [364]:
from datetime import datetime
date_format = "%Y-%m-%d"
a = datetime.strptime(temp5['day'], date_format)
b = datetime.strptime(temp6["day"], date_format)
delta = b - a
delta

TypeError: strptime() argument 1 must be str, not Series

In [369]:
from datetime import datetime
from dateutil import relativedelta

In [370]:
date_1 = '2020-12-19'
date_2 = '2021-11-07'

In [375]:
start = datetime.strptime(date_1, "%Y-%m-%d")
end =   datetime.strptime(date_2, "%Y-%m-%d")

TypeError: strptime() argument 1 must be str, not Series

In [372]:
diff = relativedelta.relativedelta(end, start)

In [373]:
diff_in_months = diff.months + diff.years * 12

In [374]:
diff_in_months

10

In [6]:
def date_convert(date_to_convert):
     return datetime.datetime.strptime(date_to_convert, '%b %d %Y').strftime('%m/%d/%Y')

In [2]:
from datetime import datetime
from dateutil import relativedelta

In [7]:
temp5["day"].apply(date_convert)

NameError: name 'temp5' is not defined

In [4]:
start = datetime.strptime(temp5["day"], "%d/%m/%Y")
end =   datetime.strptime(temp6["day"], "%d/%m/%Y")

NameError: name 'temp5' is not defined

In [9]:
temp5["day"].map(datetime.datetime.date) 

NameError: name 'temp5' is not defined

In [10]:
temp5["day"].datetime.date

NameError: name 'temp5' is not defined